In [125]:
import pandas as pd
import os
from datetime import datetime
def transform_id(id_csv): #Function to swap the ID_NOTICE_CAN field so it aligns with the one used in the xml format
    year_part = str(id_csv)[:4]
    id_xml = str(id_csv)[4:].zfill(6) + '-' + year_part
    return id_xml

def bool_converter(value):
    if pd.isnull(value) or value =='':
        return False 
    elif value == 'Y' or value == '1':
        return True
    elif value == 'N' or value == '0':
        return False
    else:
        raise ValueError(f"Unexpected value '{value}' found in the column.")
def int_converter(value):
    if pd.isnull(value) or value =='':
        return -1
    else:
        return int(value)
def lots_converter(value):
    if pd.isnull(value) or value == '' or pd.isna(value):
        return 0
    elif value.isdigit():
        return int(value) 
    else:
        raise ValueError(f"Unexpected value '{value}' found in the column.")
def date_converter(value):
    return datetime.strptime(value, '%d/%m/%y')

def flatten_csv(folder_path):

    
    columns_can_level = ['ID_NOTICE_CAN', 'TED_NOTICE_URL', 'YEAR', 'ID_TYPE', 'DT_DISPATCH', 'XSD_VERSION',
                         'CANCELLED',
                         'CORRECTIONS', 'B_MULTIPLE_CAE', 'CAE_NAME', 'CAE_NATIONALID', 'CAE_ADDRESS', 'CAE_TOWN',
                         'CAE_POSTAL_CODE', 'CAE_GPA_ANNEX', 'ISO_COUNTRY_CODE', 'ISO_COUNTRY_CODE_GPA',
                         'B_MULTIPLE_COUNTRY',
                         'ISO_COUNTRY_CODE_ALL', 'CAE_TYPE', 'EU_INST_CODE', 'MAIN_ACTIVITY',
                         'B_ON_BEHALF', 'B_INVOLVES_JOINT_PROCUREMENT', 'B_AWARDED_BY_CENTRAL_BODY', 'TYPE_OF_CONTRACT',
                         'B_FRA_AGREEMENT', 'FRA_ESTIMATED', 'B_DYN_PURCH_SYST', 'CPV', 'MAIN_CPV_CODE_GPA',
                         'B_GPA', 'GPA_COVERAGE', 'LOTS_NUMBER', 'VALUE_EURO', 'VALUE_EURO_FIN_1', 'VALUE_EURO_FIN_2',
                         'TOP_TYPE',
                         'B_ACCELERATED', 'OUT_OF_DIRECTIVES', 'B_ELECTRONIC_AUCTION', 'NUMBER_AWARDS']
    dtypes = {
        'ID_NOTICE_CAN': 'str',
        'TED_NOTICE_URL': 'str',
        'XSD_VERSION': 'str',
        'CANCELLED': 'bool',
        'CAE_NAME': 'str',
        'CAE_NATIONALID': 'str',
        'CAE_ADDRESS': 'str',
        'CAE_TOWN': 'str',
        'CAE_POSTAL_CODE': 'str',
        'CAE_GPA_ANNEX': 'str',
        'ISO_COUNTRY_CODE': 'str',
        'ISO_COUNTRY_CODE_GPA': 'str',
        'ISO_COUNTRY_CODE_ALL': 'str',
        'CAE_TYPE': 'str',
        'EU_INST_CODE': 'str',
        'MAIN_ACTIVITY': 'str',
        'TYPE_OF_CONTRACT': 'str',
        'FRA_ESTIMATED': 'str',
        'VALUE_EURO': 'float',
        'VALUE_EURO_FIN_1': 'float',
        'VALUE_EURO_FIN_2': 'float',
        'TOP_TYPE': 'str'
    }
    bool_cols = ['B_MULTIPLE_CAE', 'B_MULTIPLE_COUNTRY', "B_ON_BEHALF", "B_INVOLVES_JOINT_PROCUREMENT",
                 "B_AWARDED_BY_CENTRAL_BODY", "B_FRA_AGREEMENT", "B_DYN_PURCH_SYST", "B_GPA", "B_ACCELERATED",
                 "B_ELECTRONIC_AUCTION","OUT_OF_DIRECTIVES"]
    int_cols = ['GPA_COVERAGE','CPV','CORRECTIONS','ID_TYPE','YEAR','MAIN_CPV_CODE_GPA']
    converters = {col: bool_converter for col in bool_cols}
    converters.update({col: int_converter for col in int_cols})
    converters['LOTS_NUMBER'] = lots_converter
    converters['DT_DISPATCH'] = date_converter
    dfs = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv') and file_name.startswith('export_CAN'):
            file_path = os.path.join(folder_path, file_name)
            current_df = pd.read_csv(file_path, usecols=columns_can_level, dtype=dtypes,converters=converters)
            dfs.append(current_df)
    df = pd.concat(dfs, ignore_index=True)

    df_flat = df.groupby('ID_NOTICE_CAN').first()
    df_flat.index = df_flat.index.to_series().apply(transform_id)
    return df_flat



In [126]:
df = flatten_csv("C:/Users/afont/OneDrive/Escritorio/PROCURE/Data/TED")
df.dtypes

TED_NOTICE_URL                          object
YEAR                                     int64
ID_TYPE                                  int64
DT_DISPATCH                     datetime64[ns]
XSD_VERSION                             object
CANCELLED                                 bool
CORRECTIONS                              int64
B_MULTIPLE_CAE                            bool
CAE_NAME                                object
CAE_NATIONALID                          object
CAE_ADDRESS                             object
CAE_TOWN                                object
CAE_POSTAL_CODE                         object
CAE_GPA_ANNEX                           object
ISO_COUNTRY_CODE                        object
ISO_COUNTRY_CODE_GPA                    object
B_MULTIPLE_COUNTRY                        bool
ISO_COUNTRY_CODE_ALL                    object
CAE_TYPE                                object
EU_INST_CODE                            object
MAIN_ACTIVITY                           object
B_ON_BEHALF  

In [128]:
df.fillna({'VALUE_EURO' : -1.,
           'VALUE_EURO_FIN_1' : -1.,
           'VALUE_EURO_FIN_2' : -1.,},inplace=True)
def calculate_statistics(df, c, dtype):
    common_stats = {
        'Column': c,
        'Type': dtype,
        'Range': f"[{df[c].min()} , {df[c].max()}]" if dtype in ["Integer", "Float","Date"] else df[c].unique(),
        'Range Count': df[c].nunique(),
        'Count': df[c].count(),
        'NaN Count': df[c].isna().sum(),
    }
    
    if dtype == "Integer":
        error_values = df[c][df[c].apply(lambda x: not isinstance(x, int))]
        common_stats.update({
            'Mean': df[c].mean(),
            'Standard Deviation': df[c].std(),
            'Type Error Range': error_values.unique(),
            'Type Error Count': (df[c].apply(lambda x: not isinstance(x, int))).sum()
        })
    elif dtype == "Float":
        error_values = df[c][df[c].apply(lambda x: not isinstance(x, float))]
        common_stats.update({
            'Mean': df[c].mean(),
            'Standard Deviation': df[c].std(),
            'Type Error Range': error_values.unique(),
            'Type Error Count': (df[c].apply(lambda x: not isinstance(x, float))).sum()
        })
    elif dtype == "Boolean":
        error_values = df[c][df[c].apply(lambda x: not isinstance(x, bool))]
        common_stats.update({
            'Mean': df[c].mean(),
            'Standard Deviation': None,
            'Type Error Range': error_values.unique(),
            'Type Error Count': (df[c].apply(lambda x: not isinstance(x, bool))).sum() 
        })
    elif dtype == "String":
        error_values = df[c][df[c].apply(lambda x: not isinstance(x, str))]
        common_stats.update({
            'Mean': None,
            'Standard Deviation': None,
            'Type Error Range': error_values.unique(),
            'Type Error Count': (df[c].apply(lambda x: not isinstance(x, str))).sum() - df[c].isna().sum()
        })
    elif dtype == "Date":
        error_values = df[c][df[c].apply(lambda x: not isinstance(x, datetime))]
        common_stats.update({
            'Mean': None,
            'Standard Deviation': None,
            'Type Error Range': error_values.unique(),
            'Type Error Count': (df[c].apply(lambda x: not isinstance(x, datetime))).sum()
        })
        
    return common_stats

characteristics = []

for c in df.columns:
    if df[c].dtype == 'int32' or df[c].dtype == 'int64':
        characteristics.append(calculate_statistics(df, c, "Integer"))
    elif df[c].dtype == 'float64':
        characteristics.append(calculate_statistics(df, c, "Float"))
    elif df[c].dtype == 'bool':
        characteristics.append(calculate_statistics(df, c, "Boolean"))
    elif df[c].dtype == 'object':
        characteristics.append(calculate_statistics(df, c, "String"))
    elif df[c].dtype == 'datetime64[ns]':
        characteristics.append(calculate_statistics(df, c, "Date"))

# Convert the list of dictionaries to a DataFrame
characteristics_df = pd.DataFrame(characteristics)

# Display the resulting DataFrame
display(characteristics_df)


,Column,Type,Range,Range Count,Count,NaN Count,Mean,Standard Deviation,Type Error Range,Type Error Count
0,TED_NOTICE_URL,String,[ted.europa.eu/udl?uri=TED:NOTICE:100005-2018:...,1660226,1660226,0,NaN,NaN,[],0
1,YEAR,Integer,"[2018 , 2023]",6,1660226,0,2.020642e+03,1.693740e+00,[],0
2,ID_TYPE,Integer,"[3 , 25]",7,1660226,0,3.824234e+00,3.146108e+00,[],0
3,DT_DISPATCH,Date,"[2017-12-01 00:00:00 , 2023-12-27 00:00:00]",2198,1660226,0,NaN,NaN,[],0
4,XSD_VERSION,String,"[R209.S2, R208.S3, R208.S4, R209.S3, R209.S4, ...",7,1660226,0,NaN,NaN,[],0
5,CANCELLED,Boolean,"[False, True]",2,1660226,0,1.204655e-05,NaN,[],0
6,CORRECTIONS,Integer,"[0 , 8]",8,1660226,0,3.011036e-03,5.779688e-02,[],0
7,B_MULTIPLE_CAE,Boolean,"[False, True]",2,1660226,0,2.785283e-02,NaN,[],0
8,CAE_NAME,String,"[BIR Avfallsenergi AS, Statens Vegvesen, Reykj...",199701,1660226,0,NaN,NaN,[],0
9,CAE_NATIONALID,String,"[985 825 408, 971 032 081, None, 985 042 667, ...",66250,901878,758348,NaN,NaN,[None],0


In [150]:
index_settings="{\n\t\"mappings\": {\n\t\t\"properties\": {\n\t\t\t"
for _, row in characteristics_df.iterrows():
    index_settings = index_settings + "\"" + row['Column'] + "\" : {\"type\" : "
    if row['Type'] == 'Integer':
        index_settings = index_settings + "\"integer\"},\n\t\t\t"
    elif row['Type'] == 'Float':
        index_settings = index_settings + "\"float\" , \"null_value\": -1},\n\t\t\t"
    elif row['Type'] == 'Boolean':
        index_settings = index_settings + "\"boolean\"},\n\t\t\t"
    elif row['Type'] == 'Date':
        index_settings = index_settings + "\"date\"},\n\t\t\t"
    elif row['Type'] == 'String' and row['Range Count'] < 1000:
        index_settings = index_settings + "\"keyword\", \"null_value\": \"None\"},\n\t\t\t"
    else: 
        index_settings = index_settings + "\"text\"},\n\t\t\t"
index_settings = index_settings[:-5] + "\n\t\t}\n\t}\n}"

print(index_settings)

{
	"mappings": {
		"properties": {
			"TED_NOTICE_URL" : {"type" : "text"},
			"YEAR" : {"type" : "integer"},
			"ID_TYPE" : {"type" : "integer"},
			"DT_DISPATCH" : {"type" : "date"},
			"XSD_VERSION" : {"type" : "keyword", "null_value": "None"},
			"CANCELLED" : {"type" : "boolean"},
			"CORRECTIONS" : {"type" : "integer"},
			"B_MULTIPLE_CAE" : {"type" : "boolean"},
			"CAE_NAME" : {"type" : "text"},
			"CAE_NATIONALID" : {"type" : "text"},
			"CAE_ADDRESS" : {"type" : "text"},
			"CAE_TOWN" : {"type" : "text"},
			"CAE_POSTAL_CODE" : {"type" : "text"},
			"CAE_GPA_ANNEX" : {"type" : "keyword", "null_value": "None"},
			"ISO_COUNTRY_CODE" : {"type" : "keyword", "null_value": "None"},
			"ISO_COUNTRY_CODE_GPA" : {"type" : "keyword", "null_value": "None"},
			"B_MULTIPLE_COUNTRY" : {"type" : "boolean"},
			"ISO_COUNTRY_CODE_ALL" : {"type" : "keyword", "null_value": "None"},
			"CAE_TYPE" : {"type" : "keyword", "null_value": "None"},
			"EU_INST_CODE" : {"type" : "keyword", "null_value